## Initial imports and constants used in all cells

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
import networkx as nx
import os, ntpath, json
import ringo

FINAL_DF_NAME = 'benchmark_stats.csv'

SI_DATA_DIR = './si_data'
TESTSET_OVERVIEW_CSV = os.path.join(SI_DATA_DIR, 'testset_overview.csv')

METHOD_NAMES = {
    'ringointel': 'Ringo',
    'rdkit': 'RDKit',
    'mtd': 'XTB MTD',
    'mmbasic': 'MacroModel MCMM',
    'mmring': 'MacroModel MCS',
    'crest': 'CREST',
}
ALL_METHODS = ['ringointel', 'rdkit', 'mtd']
FULL_CASES_ORDER = ['pdb_3M6G', 'csd_FINWEE10', 'pdb_2IYA', 'csd_YIVNOG', 'pdb_1NWX', 'csd_RULSUN', 'csd_MIWTER', 'pdb_2C6H', 'csd_RECRAT', 'pdb_2QZK']

PLOT_MODES = {
    'low_energy': {
        'timing_compute': lambda df: df['time'] / df['n_lower_unique'] * df['thread_avg'],
        'time_cutoff': 100.0, # seconds per conformer
        'y_caption': 'Time per unique\nlow-energy conformer, s',
        'x_caption_new': 'Time per low-energy conformer by alternative method, s',
        'y_caption_new': 'Time per low-energy conformer by MCR, s',
    },
    'any_energy': {
        'timing_compute': lambda df: df['time'] / (df['n_lower_unique'] + df['n_higher_unique']) * df['thread_avg'],
        'time_cutoff': 10.0, # seconds per conformer
        'y_caption': 'Time per unique\nconformer, s',
        'x_caption_new': 'Time per conformer by alternative method, s',
        'y_caption_new': 'Time per conformer by MCR, s',
    },
}

## Generate **bar** plots of sampling rates

In [ ]:
def plot_df(df, y_caption, result_file, show_legend, time_cutoff=None, **kwargs):
    from plotnine import ggplot, aes, labs, scale_fill_manual, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_bar, position_dodge, ylim
    normal_theme = (theme_bw() +
                    theme(panel_grid_major = element_blank(),
                        panel_grid_minor = element_blank(),
                        panel_border = element_rect(colour='black', fill=None, size=1),
                        axis_line = element_line(colour='black'),
                        axis_title = element_text(size=16, face='bold', ma='center'),
                        axis_text = element_text(size=14),
                        legend_title = element_text(size=14, face='bold'),
                        legend_text = element_text(size=14),
                        figure_size=(10, 4)))

    # Colors from Set2
    colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#a65628", "#f781bf"]

    plot = ggplot(df, aes(y='time_per_conf', x='testcase', fill='Sampling method')) + \
        geom_bar(stat='identity', position=position_dodge(width=0.9)) + \
        labs(y=y_caption, x='') + normal_theme + \
        theme(axis_text_x = element_text(angle = 60)) + scale_fill_manual(values=colorscheme)
    if time_cutoff is not None:
        plot += ylim(0, time_cutoff)
    if not show_legend:
        plot += theme(legend_position="none")
    
    print(plot)
    plot.save(result_file, verbose=False)

def plot_ten_molecules(timing_compute, **kwargs):
    df = pd.read_csv(FINAL_DF_NAME)
    df['time_per_conf'] = timing_compute(df)
    df['method'] = df['method'].replace(METHOD_NAMES)
    method_type = CategoricalDtype(categories=[method_name for method_name in METHOD_NAMES.values()], ordered=True)
    df['method'] = df['method'].astype(method_type)    
    
    # Use predefined ordering of testcases on the plot
    testcase_type = CategoricalDtype(categories=FULL_CASES_ORDER, ordered=True)
    df['testcase'] = df['testcase'].astype(testcase_type)
    df = df[df['testcase'].isin(FULL_CASES_ORDER)]

    df = df.rename(columns={'method': 'Sampling method'})

    df['testcase'] = df['testcase'].str.replace('csd_', '')
    df['testcase'] = df['testcase'].str.replace('pdb_', '')
    testcase_type = CategoricalDtype(categories=[x.replace('csd_', '').replace('pdb_', '') for x in FULL_CASES_ORDER], ordered=True)
    df['testcase'] = df['testcase'].astype(testcase_type)

    # Cut bar heights
    if 'time_cutoff' in kwargs:
        df['time_per_conf'] = df['time_per_conf'].apply(lambda x: kwargs['time_cutoff'] if x > kwargs['time_cutoff'] else x)

    plot_df(df, **kwargs)

def plot_all_molecules(timing_compute, **kwargs):
    df = pd.read_csv(FINAL_DF_NAME)
    df['time_per_conf'] = timing_compute(df)
    # Cut the bar heights if required
    if 'time_cutoff' in kwargs:
        df['time_per_conf'] = df['time_per_conf'].apply(lambda x: kwargs['time_cutoff'] if x > kwargs['time_cutoff'] else x)
    
    df = df[df['method'].isin(ALL_METHODS)]
    df['method'] = df['method'].replace(METHOD_NAMES)
    method_type = CategoricalDtype(categories=[method_name for method_name in METHOD_NAMES.values()], ordered=True)
    df['method'] = df['method'].astype(method_type)    
    
    # df = df[~df['testcase'].isin(FULL_CASES_ORDER)]
    df['testcase'] = df['testcase'].str.replace('csd_', '')
    df['testcase'] = df['testcase'].str.replace('pdb_', '')
    order_df = df.loc[df['method'] == METHOD_NAMES['ringointel']].sort_values(by=['time_per_conf'], ascending=False)
    order_df = order_df.reset_index()
    cat_type = CategoricalDtype(categories=order_df['testcase'], ordered=True)

    df = df.rename(columns={'method': 'Sampling method'})
    df = df.reset_index()

    
    TIMES = [0.1, 1.0, 10.0, 30.0]
    METHODS = df['Sampling method'].unique()
    overview_df = pd.read_csv(TESTSET_OVERVIEW_CSV)
    overview_df['testcase'] = overview_df['testcase'].str.replace('csd_', '')
    overview_df['testcase'] = overview_df['testcase'].str.replace('pdb_', '')
    VARIABLE = 'Number of kinematic DOFs'
    
    for method in METHODS:
        print(f'-{method}-')
        cur_df = df[df['Sampling method'] == method]
        cur_df = cur_df.reset_index()
        cur_df = cur_df.merge(overview_df, on='testcase', how='left')
        cur_df[VARIABLE] = cur_df[VARIABLE].astype(float)
        for top_time in TIMES:
            sub_df = cur_df[cur_df['time_per_conf'] < top_time]
            other_df = cur_df[cur_df['time_per_conf'] > top_time]
            # print(sub_df)
            print(f"<{top_time} = {len(sub_df)} ({other_df[VARIABLE].mean()})")

    # step_size = 20
    # for i in range(0, len(order_df['testcase']), step_size):
    #     current_testcases = order_df['testcase'][i : i + step_size]
    #     cur_df = df[df['testcase'].isin(current_testcases)].copy()
    #     cur_df['testcase'] = cur_df['testcase'].astype(cat_type)
    #     cur_kwargs = {
    #         **kwargs,
    #         'result_file': kwargs['result_file'].format(plot_type=i)
    #     }
    #     plot_df(cur_df, **cur_kwargs)


In [ ]:
# Plots of figures for main text
# Hatches were added by hand in Adobe Illustrator
for plotting_mode, basic_kwargs in PLOT_MODES.items():
    print(f'============\n{plotting_mode}\n============')
    plotting_kwargs = {
        **basic_kwargs,
        'show_legend': True,
        'result_file': os.path.join(SI_DATA_DIR, f'{plotting_mode}_timings.svg'),
    }
    plot_ten_molecules(**plotting_kwargs)

In [ ]:
# Plots of figures for SI
# Hatches were added by hand in Adobe Illustrator
for plotting_mode, basic_kwargs in PLOT_MODES.items():
    print(f'============\n{plotting_mode}\n============')
    plotting_kwargs = {
        **basic_kwargs,
        'show_legend': False,
        'result_file': os.path.join(SI_DATA_DIR, f'{plotting_mode}_si.svg'),
    }
    # del plotting_kwargs['time_cutoff']
    # plot_ten_molecules(**plotting_kwargs)

    plotting_kwargs['result_file'] = os.path.join(SI_DATA_DIR, '%s_{plot_type}.png' % plotting_mode)
    plot_all_molecules(**plotting_kwargs)

## Generate **point** plots of sampling rates

In [ ]:
def plot_df(df, y_caption_new, x_caption_new, result_file, show_legend, time_cutoff=None, **kwargs):
    from plotnine import ggplot, geom_abline, geom_line, aes, labs, facet_grid, scale_color_manual, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_point, position_dodge, ylim, scale_x_log10, scale_y_log10, scale_color_gradient
    normal_theme = (theme_bw() +
                    theme(panel_grid_major = element_blank(),
                        panel_grid_minor = element_blank(),
                        panel_border = element_rect(colour='black', fill=None, size=1),
                        axis_line = element_line(colour='black'),
                        axis_title = element_text(size=16, face='bold', ma='center'),
                        axis_text = element_text(size=14),
                        legend_title = element_text(size=14, face='bold'),
                        legend_text = element_text(size=14),
                        figure_size=(8, 8)))

    # Colors from Set2 "#e41a1c",
    colorscheme = ["#377eb8", "#4daf4a", "#984ea3", "#a65628", "#f781bf"]
    # colorscheme = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a']

    plot = ggplot(df, aes(x='time_per_conf', y='ringo_time', color='Sampling_method')) + \
        normal_theme + \
        geom_abline(intercept=0, slope=1, linetype='solid') + \
        geom_abline(intercept=-1, slope=1, linetype='dashdot') + \
        geom_abline(intercept=-2, slope=1, linetype='dashed') + \
        geom_abline(intercept=-3, slope=1, linetype='dotted') + \
        geom_point(size=2, shape='X') + \
        scale_x_log10() + \
        scale_y_log10() + \
        labs(x = x_caption_new, y = y_caption_new) + \
        scale_color_manual(values=colorscheme) + facet_grid('Sampling_method~.')# + \
        # geom_point(size=4, shape='X') + \
        # 
    if not show_legend:
        plot += theme(legend_position="none")
    
    print(plot)
    plot.save(result_file, verbose=False)

def plot_ten_molecules(timing_compute, all_molecules=False, **kwargs):
    df = pd.read_csv(FINAL_DF_NAME)
    df['time_per_conf'] = timing_compute(df)
    df['correct_time'] = df['time'] * df['thread_avg']
    
    if all_molecules:
        df = df[df['method'].isin(ALL_METHODS)]
    df['method'] = df['method'].replace(METHOD_NAMES)
    
    # Use predefined ordering of testcases on the plot
    if not all_molecules:
        testcase_type = CategoricalDtype(categories=FULL_CASES_ORDER, ordered=True)
        df['testcase'] = df['testcase'].astype(testcase_type)
        df = df[df['testcase'].isin(FULL_CASES_ORDER)]

    ringo_df = df[df['method'] == METHOD_NAMES['ringointel']]
    ringo_df = ringo_df.rename(columns={'time_per_conf': 'ringo_time', 'correct_time': 'ringo_corr_time'})[['testcase', 'ringo_time', 'ringo_corr_time']]
    df = df[~(df['method'] == METHOD_NAMES['ringointel'])]
    df = df.merge(ringo_df, on='testcase', how='left')
    assert METHOD_NAMES['ringointel'] not in df['method'].unique()
    
    method_type = CategoricalDtype(categories=[method_name for method_name in METHOD_NAMES.values() 
                                               if method_name != METHOD_NAMES['ringointel']], ordered=True)
    df['method'] = df['method'].astype(method_type)    
    
    df = df.rename(columns={'method': 'Sampling_method'})
    df['testcase'] = df['testcase'].str.replace('csd_', '')
    df['testcase'] = df['testcase'].str.replace('pdb_', '')
    if not all_molecules:
        testcase_type = CategoricalDtype(categories=[x.replace('csd_', '').replace('pdb_', '') for x in FULL_CASES_ORDER], ordered=True)
        df['testcase'] = df['testcase'].astype(testcase_type)
    df = df.reset_index(drop=True)

    df['accel'] = df['correct_time'] / df['ringo_corr_time']
    for method in df['Sampling_method'].unique():
        sub_df = df[df['Sampling_method'] == method].copy()
        sub_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        sub_df.dropna(inplace=True)
        
        avg_gen_rate = (sub_df['time_per_conf'] / sub_df['ringo_time']).mean()
        min_acceleration = sub_df['accel'].min()
        max_acceleration = sub_df['accel'].max()
        print(f'Outperformed {method} by {avg_gen_rate} times ({min_acceleration} - {max_acceleration})')

    plot_df(df, **kwargs)

In [ ]:
# Plots of figures for main text
for plotting_mode, basic_kwargs in PLOT_MODES.items():
    print(f'============\n{plotting_mode}\n============')
    plotting_kwargs = {
        **basic_kwargs,
        'show_legend': True,
        'result_file': os.path.join(SI_DATA_DIR, f'{plotting_mode}_timings.svg'),
    }
    plot_ten_molecules(**plotting_kwargs)

In [ ]:
# Plots of figures for SI
# Hatches were added by hand in Adobe Illustrator
for plotting_mode, basic_kwargs in PLOT_MODES.items():
    print(f'============\n{plotting_mode}\n============')
    plotting_kwargs = {
        **basic_kwargs,
        'show_legend': True,
        'result_file': os.path.join(SI_DATA_DIR, f'{plotting_mode}_total_si.svg'),
    }
    del plotting_kwargs['time_cutoff']
    # plot_ten_molecules(**plotting_kwargs)

    plotting_kwargs['result_file'] = os.path.join(SI_DATA_DIR, '%s_total_si.svg' % plotting_mode)
    plot_ten_molecules(all_molecules=True, **plotting_kwargs)

## Generate CSVs for SI with testset overview and benchmark data

In [ ]:
from charges import CHARGES
    
TESTSET_JSON = 'testcases.json'
with open(TESTSET_JSON, 'r') as f:
    sdf_mapping = json.load(f)

df = pd.read_csv(FINAL_DF_NAME)
# Load testset overview
testcases = df['testcase'].unique()

# Print the structure of testset
testcases_list = list(testcases)
testset_parts = set(name.split('_')[0] for name in testcases_list)
for part_idx, testset_part in enumerate(testset_parts):
    molecule_ids = [name.split('_')[1] for name in testcases_list if name.startswith(testset_part)]
    print(f'{part_idx+1}) {testset_part.upper()}: {", ".join(sorted(molecule_ids))}')

# Generate overview
overview_df = pd.DataFrame()
overview_df['testcase'] = testcases
def get_overview(row):
    molname = row['testcase']

    m = ringo.Molecule(sdf_mapping[molname])
    result = ringo.get_molecule_statistics(m)
    result['composition'] = ''.join([f'{element}{count}' for element, count in result['composition'].items()])
    
    if molname in CHARGES:
        charge = CHARGES[molname]
    else:
        charge = 0
    result['charge'] = charge

    return pd.Series(result)
new_columns = overview_df.apply(get_overview, axis=1)
overview_df = pd.concat([overview_df, new_columns], axis=1)
overview_df = overview_df.rename(columns={
    'method': 'Sampling method',
    'composition': 'Composition',
    'charge': 'Total charge',
    'num_atoms': 'Number of atoms',
    'num_heavy_atoms': 'Number of heavy atoms',
    'num_bonds': 'Number of bonds',
    'num_rotatable_bonds': 'Number of rotatable bonds',
    'num_cyclic_rotatable_bonds': 'Number of cyclic rotatable bonds',
    'n_flexible_rings': 'Number of flexible cycles',
    'n_rigid_rings': 'Number of rigid cycles',
    'largest_macrocycle_size': 'Largest macrocycle size',
    'num_dofs': 'Number of kinematic DOFs',
    'cyclomatic_number': 'Cyclomatic number (n_edges-n_atoms+1)',
})
overview_df.to_csv(TESTSET_OVERVIEW_CSV, index=False)

# Load benchmark data
for item in PLOT_MODES.values():
    df[item['y_caption']] = item['timing_compute'](df)
df['method'] = df['method'].replace(METHOD_NAMES)
df = df.drop(columns=[
    'n_failed_opts',
])
df = df.rename(columns={
    'method': 'Sampling method',
    'n_total_generated': 'Conformations generated',
    'n_unique_generated': 'Unique conformers generated',
    'n_duplicates': 'Duplicates after optimization',
    'n_failed_topo': 'Incorrect topology after optimization',
    'n_higher_unique': 'High-energy conformations (>15 kcal/mol)',
    'n_lower_unique': 'Low-energy conformations (<15 kcal/mol)',
})
df.to_csv(os.path.join(SI_DATA_DIR, 'timings_results.csv'), index=False)

## Compare MC against MCR

#### Compare frequencies of zero solutions

In [ ]:
CURRENT_TESTSET = 'ringsZero'
# CURRENT_TESTSET = 'rings'

df = pd.concat([
    pd.read_csv(csvfile)
    for csvfile in (f'ringoMC_{CURRENT_TESTSET}_mcmode.csv', f'ringoMCR_{CURRENT_TESTSET}_mcmode.csv')
], ignore_index=True)

df['success_rate'] = df['nsucc']/df['ntries']*100
df = df[df['method'] == 'ringoMC']
df['num_rings'] = df['testcase'].apply(lambda x: int(x.split('_')[1]))
print(df)

from plotnine import ggplot, geom_abline, aes, labs, scale_color_gradientn, geom_line, scale_y_continuous, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_point, position_dodge, ylim, scale_x_log10, scale_y_log10, scale_color_gradient
normal_theme = (theme_bw() +
                theme(panel_grid_major = element_blank(),
                    panel_grid_minor = element_blank(),
                    panel_border = element_rect(colour='black', fill=None, size=1),
                    axis_line = element_line(colour='black'),
                    axis_title = element_text(size=16, face='bold', ma='center'),
                    axis_text = element_text(size=14),
                    legend_title = element_text(size=14, face='bold'),
                    legend_text = element_text(size=14),
                    figure_size=(8, 4)))

# Colors from Set2
colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#a65628", "#984ea3"]

COLUMN_RENAME = {
    'success_rate': 'Success rate, %',
    'num_rings': 'Number of rings          (n)',
}
def exponential_function(x, a, b):
    return a * np.exp(-b * x)
from scipy.optimize import curve_fit
params, covariance = curve_fit(exponential_function, df['num_rings'], df['success_rate'], maxfev=10000)
a_best, b_best = params
x_values = np.arange(df['num_rings'].min(), df['num_rings'].max() + 0.1, 0.1)
y_values = exponential_function(x_values, a_best, b_best)
ref_df = pd.DataFrame({'x': x_values, 'y': y_values})

plotnine_adds = []
plotnine_adds.append(geom_line(ref_df, aes(y='y', x='x'), size=1.0, color='#AAAAAA'))
plotnine_adds.append(geom_abline(intercept=0, slope=0, linetype='dashed'))
df = df.rename(columns=COLUMN_RENAME)

print(df)
plot = ggplot() + plotnine_adds + \
    geom_point(df, aes(y=COLUMN_RENAME['success_rate'], x=COLUMN_RENAME['num_rings'], color=COLUMN_RENAME['num_rings']), size=2) + scale_y_continuous(limits=(0, None)) + \
    normal_theme + scale_color_gradientn(colors=['green', 'orange', 'red']) + labs(x=COLUMN_RENAME['num_rings'], y=COLUMN_RENAME['success_rate'])
print(plot)
plot.save(os.path.join(SI_DATA_DIR, 'ringoMC_zerosoln_succrate.svg'), verbose=False)

# for method in df['method'].unique():
#     print(f'==== {method} ====')
#     sub_df = df[df['method'] == method]
#     for testcase in sorted(sub_df['testcase'].unique()):
#         item = sub_df[sub_df['testcase'] == testcase].iloc[0]
#         num_rings = int(testcase.split('_')[1])
#         print(f'Zero freq for {testcase} = {item["nzero"]/item["ntries"]} Theoretical={1-2**(-num_rings)}')

#### Compare frequencies of overlaps for non-cyclic test molecules

In [ ]:
CURRENT_TESTSET = 'noncyclics'

df = pd.concat([
    pd.read_csv(csvfile)
    for csvfile in (f'ringoMC_{CURRENT_TESTSET}_mcmode.csv', f'ringoMCR_{CURRENT_TESTSET}_mcmode.csv')
], ignore_index=True)


df['success_rate'] = df['nsucc'] / df['ntries'] * 100
df = df[df['method'] == 'ringoMC']
df['num_rings'] = df['testcase'].apply(lambda x: int(x.split('_')[1]))
print(df)

from plotnine import ggplot, geom_abline, aes, labs, scale_color_manual, geom_line, scale_color_gradientn, scale_y_continuous, scale_colour_distiller, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_point, position_dodge, ylim, scale_x_log10, scale_y_log10, scale_color_gradient
normal_theme = (theme_bw() +
                theme(panel_grid_major = element_blank(),
                    panel_grid_minor = element_blank(),
                    panel_border = element_rect(colour='black', fill=None, size=1),
                    axis_line = element_line(colour='black'),
                    axis_title = element_text(size=16, face='bold', ma='center'),
                    axis_text = element_text(size=14),
                    legend_title = element_text(size=14, face='bold'),
                    legend_text = element_text(size=14),
                    figure_size=(8, 4)))

# Colors from Set2
colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#a65628", "#984ea3"]
# print(df)

# For linear only
def exponential_function(x, a, b):
    return a * np.exp(-b * x)
df['num_rings'] = df['num_rings'] + 1
COLUMN_RENAME = {
    'success_rate': 'Success rate, %',
    'num_rings': 'Number of CH₂ groups (n)',
}
plotnine_adds = []
plotnine_adds.append(geom_abline(intercept=0, slope=0, linetype='dashed'))

from scipy.optimize import curve_fit
params, covariance = curve_fit(exponential_function, df['num_rings'], df['success_rate'], maxfev=10000)
a_best, b_best = params
x_values = np.arange(df['num_rings'].min(), df['num_rings'].max(), 0.1)
y_values = exponential_function(x_values, a_best, b_best)
ref_df = pd.DataFrame({'x': x_values, 'y': y_values})
df = df.rename(columns=COLUMN_RENAME)
plotnine_adds.append(geom_line(ref_df, aes(y='y', x='x'), size=1.0, color='#AAAAAA'))

plot = ggplot() + plotnine_adds + \
    geom_point(df, aes(y=COLUMN_RENAME['success_rate'], x=COLUMN_RENAME['num_rings'], color=COLUMN_RENAME['num_rings']), size=2) + \
    normal_theme + \
    scale_color_gradientn(colors=['green', 'orange', 'red']) + scale_y_continuous(limits=(0, None)) + \
    labs(x=COLUMN_RENAME['num_rings'], y=COLUMN_RENAME['success_rate'])
print(plot)
plot.save(os.path.join(SI_DATA_DIR, 'ringoMC_overlap_succrate.svg'), verbose=False)

#### Plot generation rates

In [ ]:
CURRENT_TESTSET = 'main'

df = pd.concat([
    pd.read_csv(csvfile)
    for csvfile in (f'ringoMC_{CURRENT_TESTSET}_mcmode.csv', f'ringoMCR_{CURRENT_TESTSET}_mcmode.csv')
], ignore_index=True)

# df['time_per_conf'] = df['time'] / df['nsucc']
df['time_per_conf'] = df['time'] / df['nunique']

METHOD_NAMES = {
    'ringoMC': 'conventional MC',
    'ringoMCR': 'MC with Refinements',
}

AXIS_NAMES = {
    key: f'Time spent by\n{value}, s'
    for key, value in METHOD_NAMES.items()
}

df['testcase'] = df['testcase'].str.replace('csd_', '')
df['testcase'] = df['testcase'].str.replace('pdb_', '')

df = df.pivot(index='testcase', columns='method', values='time_per_conf').reset_index()
df.columns.name = None  # Remove the columns' name
# df = df.rename(columns={'ringoMC': 'time_MC', 'ringoMCR': 'time_MCR'})
df = df.rename(columns=AXIS_NAMES)

if CURRENT_TESTSET == 'main':
    overview_df = pd.read_csv(TESTSET_OVERVIEW_CSV)
    overview_df['testcase'] = overview_df['testcase'].str.replace('csd_', '')
    overview_df['testcase'] = overview_df['testcase'].str.replace('pdb_', '')
    df = df.merge(overview_df, on='testcase', how='left')
    VARIABLE = 'Number of kinematic DOFs'
    df[VARIABLE] = df[VARIABLE].astype(float)
else:
    df['Number of rings'] = df['testcase'].apply(lambda x: x.split('_')[1])
    VARIABLE = 'Number of rings'


def plot_df(df):
    from plotnine import ggplot, geom_abline, aes, labs, scale_color_manual, scale_colour_distiller, scale_color_gradientn, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_point, position_dodge, ylim, scale_x_log10, scale_y_log10, scale_color_gradient
    normal_theme = (theme_bw() +
                    theme(panel_grid_major = element_blank(),
                        panel_grid_minor = element_blank(),
                        panel_border = element_rect(colour='black', fill=None, size=1),
                        axis_line = element_line(colour='black'),
                        axis_title = element_text(size=16, face='bold', ma='center'),
                        axis_text = element_text(size=14),
                        legend_title = element_text(size=14, face='bold'),
                        legend_text = element_text(size=14),
                        figure_size=(10, 4)))

    # Colors from Set2
    colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#a65628", "#984ea3"]

    # , color='Sampling method'
    print(df)
    plot = ggplot(df, aes(y=AXIS_NAMES['ringoMCR'], x=AXIS_NAMES['ringoMC'], color=VARIABLE)) + \
        geom_point(size=2) + \
        normal_theme + \
        scale_x_log10() + \
        geom_abline(intercept=0, slope=1, linetype='dashed', color='red') + \
        geom_abline(intercept=-1, slope=1, linetype='dashed', color='blue') + \
        geom_abline(intercept=-2, slope=1, linetype='dashed', color='green') + \
        scale_y_log10() + \
        scale_color_gradientn(colors=['green', 'orange', 'red']) #+ \
        # scale_colour_distiller(type="seq", palette="GnRd", direction=1)# + \
        # scale_color_manual(values=colorscheme)
        # geom_abline(intercept=0, slope=1, linetype='solid') + \
        # geom_abline(intercept=-1, slope=1, linetype='dashed') + \
        # geom_abline(intercept=-2, slope=1, linetype='dotted') + \
    
    plot.save(os.path.join(SI_DATA_DIR, f'mcmodes_{CURRENT_TESTSET}_compare.svg'), verbose=False)
    print(plot)
plot_df(df)

## Plot *Ringo* processing speeds for fused systems of different size

In [ ]:
TIMINGCSV_NAME = './si_data/ringo_speed2.csv'

df = pd.read_csv(TIMINGCSV_NAME)

df['time'] *= 1000.0
succ_subdf = df[df['exectype'] == 'succ']
zero_subdf = df[df['exectype'] == 'zero']
for testcase in sorted(df['testcase'].unique()):
    num_succ = len(succ_subdf[succ_subdf["testcase"] == testcase])
    num_zero = len(zero_subdf[zero_subdf["testcase"] == testcase])
    nrings = int(testcase.split('_')[1])
    print(f'Zero soln rate for "{testcase}" = {num_zero / (num_succ + num_zero) * 100} (Excepted = {(1-0.5**nrings)*100})')
    
for testcase in df['testcase'].unique():
    num_succ = len(succ_subdf[succ_subdf["testcase"] == testcase])
    print(f'Num samples for "{testcase}" = {num_succ}')
df = succ_subdf

avg_df = {
    'avg_time': [],
    'testcase': [],
}
for testcase in df['testcase'].unique():
    avg_time = df[df["testcase"] == testcase]['time'].mean()
    avg_df['testcase'].append(testcase)
    avg_df['avg_time'].append(avg_time)
    print(f'Avg time for "{testcase}" = {avg_time} ms')
avg_df = pd.DataFrame(avg_df)
avg_df['Number of rings'] = avg_df['testcase'].apply(lambda x: x.split('_')[1])

# sample_size = 10000
# subsamples = []
# for testcase in df['testcase'].unique():
#     subsample = df[df['testcase'] == testcase].sample(n=sample_size)
#     subsamples.append(subsample)
# df = pd.concat(subsamples)
df['Number of rings'] = df['testcase'].apply(lambda x: x.split('_')[1])
# df = df[df['Number of rings'] == '1']

from plotnine import ggplot, aes, geom_density, scale_color_manual, geom_point, xlim, theme_bw, theme, element_blank, element_line, element_text, element_rect, labs, scale_y_continuous, scale_fill_manual
normal_theme = (theme_bw() +
    theme(panel_grid_major = element_blank(),
        panel_grid_minor = element_blank(),
        panel_border = element_rect(colour='black', fill=None, size=1),
        axis_line = element_line(colour='black'),
        axis_title = element_text(size=16, face='bold', ma='center'),
        axis_text = element_text(size=14),
        legend_title = element_text(size=14, face='bold'),
        legend_text = element_text(size=14),
        figure_size=(6, 4)
    )
)
print(df)
colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#a65628", "#984ea3"]
# plot = ggplot(df, aes(x='time', fill='Number of rings')) + \
#     labs(x='Time per one set of DOFs, ms', y='Distribution density') + \
#     xlim(0, 1.0) + geom_density() + normal_theme + scale_y_continuous(breaks=None) + \
#     scale_fill_manual(values=colorscheme)
# print(plot)
# plot.save(os.path.join(SI_DATA_DIR, 'ringo_scaling.svg'), verbose=False)

plot = ggplot(avg_df, aes(x='Number of rings', y='avg_time', color='Number of rings')) + geom_point(size=3) + \
    labs(x='Number of rings', y='Average time, ms') + normal_theme + scale_color_manual(values=colorscheme) + \
    theme(figure_size=(5, 3))
print(plot)
plot.save(os.path.join(SI_DATA_DIR, 'ringo_avg_scaling.svg'), verbose=False)

# Zero
# from plotnine import ggplot, aes, geom_density, xlim
# plot = ggplot(df, aes(x='time', fill='testcase')) + xlim(0, 0.1) + geom_density()
# print(plot)

## Summary of topology corruption in ETKDG

In [ ]:
import ringo, json
from chemscripts.utils import to_xyz

OPTSTATS_JSON = 'optimization_stats.json'
RESULTING_XYZ = os.path.join(SI_DATA_DIR, 'rdkit_topofail.xyz')

with open(OPTSTATS_JSON, 'r') as f:
    ensembles_stats = json.load(f)

xyz_parts = []
look_for_status = 'topofail'
for xyzname, status_counts in ensembles_stats.items():
    if look_for_status not in status_counts:
        continue
    
    xyzname_clean = ntpath.basename(xyzname)
    parts = xyzname_clean.replace('.xyz', '').split('_')
    method = METHOD_NAMES[parts[-1]]
    molname = '_'.join(parts[:-1])
    assert 'rdkit' in xyzname_clean, 'Some method (not RDKit) produced incorrect topology! Method - ' + method

    p = ringo.Confpool()
    p.include_from_file(xyzname)
    p.filter(lambda m: look_for_status in m.descr)
    assert len(p) == status_counts[look_for_status]

    for m in p:
        xyz_parts.append(to_xyz(m.xyz, p.atom_symbols, description=f"{molname}; {method}; {m.descr}"))

with open(RESULTING_XYZ, 'w') as f:
    f.write('\n'.join(xyz_parts))


## Analyze the GFN-FF topology fails

In [ ]:
# Execute xtboptimize.py first
from tqdm import tqdm
from chemscripts.geom import Molecule
import matplotlib.pyplot as plt

LOWER_DIR = './ringointel_conformers/'
LOWER_XYZ = os.path.join(LOWER_DIR, '{molname}_ringointel.xyz')
OPTIMIZED_DIR = './xtbpreoptimized_conformers'
OPTIMIZED_XYZ = os.path.join(OPTIMIZED_DIR, '{molname}_ringointel.xyz')
FAILSUMMARY_JSON = 'xtb_failsummary.json'
TESTSET_JSON = 'testcases.json'
PATTERNS_JSON = os.path.join(SI_DATA_DIR, 'gfnff_patterns.json')


def graph_serialize(gr):
    data = {
        'nodes': {node: gr.nodes[node] for node in gr.nodes},
        'edges': [edge for edge in gr.edges],
        'edge_attrs': [gr[edge[0]][edge[1]] for edge in gr.edges],
    }
    return data

def graph_parse(data):
    # Create an empty graph
    G = nx.Graph()
    # Add nodes with attributes
    for node, attr in data['nodes'].items():
        G.add_node(int(node), **attr)
    # Add edges with attributes
    for edge, edge_attr in zip(data['edges'], data['edge_attrs']):
        G.add_edge(*edge, **edge_attr)
    return G

COLOR_MAP = {
    'broken': 'red',
    'extra': 'blue',
}
def add_edge_colors(G):
    for edge in G.edges:
        G[edge[0]][edge[1]]['color'] = COLOR_MAP[G[edge[0]][edge[1]]['type']]

def same_graphs(G1, G2):
    return nx.is_isomorphic(G1, G2, node_match=lambda n1, n2: n1 == n2, edge_match=lambda e1, e2: e1 == e2)

with open(TESTSET_JSON, 'r') as f:
    full_testset = json.load(f)
with open(FAILSUMMARY_JSON, 'r') as f:
    fail_summary = json.load(f)

In [ ]:
# Prepare record list for the following analysis
record_data = {}
for molname, conformers_data in fail_summary.items():
    record_data[molname] = {}
    for confname, faildata in conformers_data.items():
        record_data[molname][confname] = faildata
    # if len(record_data[molname]) > 0:
    #     break

# Build df of initial distances from which extra bonds were created
df_data = {
    'molname': [],
    'confname': [],
    'bond': [],
    'original_value': [],
}
for molname, conformers_data in tqdm(record_data.items()):
    original_p = ringo.Confpool()
    original_p.include_from_file(LOWER_XYZ.format(molname=molname))
    for confname, faildata in conformers_data.items():
        conf_id = int(confname.split()[1])
        if len(original_p) <= conf_id or f'Conformer {conf_id}' not in original_p[conf_id].descr:
            conf_id = None
            for i, m in enumerate(original_p):
                if confname in m.descr:
                    conf_id = i
                    break
            assert conf_id is not None, f'{confname} not in {LOWER_XYZ.format(molname=molname)}'
        for newbond in faildata['extra']:
            df_data['molname'].append(molname)
            df_data['confname'].append(confname)
            df_data['bond'].append(f'{newbond[0] + 1}-{newbond[1] + 1}')
            df_data['original_value'].append(original_p[conf_id].l(newbond[0] + 1, newbond[1] + 1))
df = pd.DataFrame(df_data)
df.to_csv(os.path.join(SI_DATA_DIR, 'length_summary.csv'), index=False)
print(f"Maximal length of extra bond = {df['original_value'].max()}")
print(f"Average length of extra bonds = {df['original_value'].mean()}")
print(f"Minimal length of extra bond = {df['original_value'].min()}")

# Save the structures from 'record_data' into XYZ-file
record_p = ringo.Confpool()
for molname, conformers_data in record_data.items():
    original_p = ringo.Confpool()
    original_p.include_from_file(LOWER_XYZ.format(molname=molname))
    optimized_p = ringo.Confpool()
    optimized_p.include_from_file(OPTIMIZED_XYZ.format(molname=molname))
    for confname, faildata in conformers_data.items():
        conf_id = int(confname.split()[1])
        assert str(conf_id) in original_p[conf_id].descr
        assert str(conf_id) in optimized_p[conf_id].descr
        comment = f"broken = {repr(tuple(f'{i+1}-{j+1}' for i, j in faildata['broken']))}"
        record_p.include_from_xyz(original_p[conf_id].xyz, f'original; {original_p[conf_id].descr}')
        record_p.include_from_xyz(optimized_p[conf_id].xyz, f'optimized; {comment}; {optimized_p[conf_id].descr}')
        record_p.atom_symbols = original_p.atom_symbols
record_p.save('check.xyz')

# Prepare data for visualization in the following cell
change_patterns = []
for molname, conformers_data in fail_summary.items():
    sdf_name = full_testset[molname]
    ccmol = Molecule(sdf=sdf_name)
    graph = ccmol.G
    
    for confname, faildata in conformers_data.items():
        change_graph = nx.Graph()
        for bond in faildata['broken'] + faildata['extra']:
            if not change_graph.has_node(bond[0]):
                change_graph.add_node(bond[0], symbol=graph.nodes[bond[0]]['symbol'])
            if not change_graph.has_node(bond[1]):
                change_graph.add_node(bond[1], symbol=graph.nodes[bond[1]]['symbol'])
        for bond_state, bonds in faildata.items():
            for bond in bonds:
                change_graph.add_edge(*bond, type=bond_state)
        
        for component in nx.connected_components(change_graph):
            subgraph = change_graph.subgraph(component)
            match_found = False
            for test_graph in change_patterns:
                if same_graphs(test_graph, subgraph):
                    match_found = True
                    break
            if not match_found:
                change_patterns.append(subgraph)
    print(len(change_patterns))
change_patterns = [
    graph_serialize(pat)
    for pat in change_patterns
]
with open(PATTERNS_JSON, 'w') as f:
    json.dump(change_patterns, f)

### Visualize possible arrangements of broken & extra bonds

In [ ]:
with open(PATTERNS_JSON, 'r') as f:
    change_patterns = json.load(f)

for i in range(40):
    G = graph_parse(change_patterns[i])
    add_edge_colors(G)

    plt.figure(figsize=(3, 2))

    # Specify node positions (optional)
    pos = nx.spring_layout(G)
    # Draw node labels
    nx.draw_networkx(G, pos, node_size=300,
        labels=nx.get_node_attributes(G, 'symbol'),
        edge_color=[G[u][v]['color'] for u, v in G.edges()],
        width=2,
        node_color='white'
    )

    # Display the graph
    plt.axis('off')
    plt.show()

## Get percentage of found clusters

In [ ]:
from diversity_analysis import get_diversity_df, RINGO_METHOD

testcase_select = [
    'pdb_2QZK',
    'csd_RECRAT',
    'pdb_2IYA',
    'pdb_1NWX',
    'pdb_3M6G',
    'pdb_2C6H',
    'csd_MIWTER'
]

diversity_analysis_settings = {
    'main_wd': './filtered_ensemble_diversity',
    'mode': 'filtered',
    'keep_unperspective': False,
    'keep_perspective': True,
    'use_refconformers': False,
    'plot_highenergy': True, # We removed them already, so this disables the redundant check for perspective conformers during plotting
    'plot_perspective': True,
    'expand_clusters': True,
    'merge_unclustered': False,
    'show_alternative_all': False,
    'show_alternative_exclusive': True,
    'show_mcr_exclusive': True,
    'report_segments': ['reachability', 'diversity'],
}

methods = ['crest', 'mmring', 'mmbasic', 'mtd', 'rdkit']
for molname in testcase_select:
    print(molname)
    df, custom_order, key_names = get_diversity_df(molname, diversity_analysis_settings)
    df = df[~df['unclustered']]
    assert not df.isna().any().any(), repr(df[df.isna()])
    
    all_clusters = set(df['cluster'].unique())
    ringo_clusters = set(df[df[f'{RINGO_METHOD}_found']]['cluster'].unique())
    
    ringo_discovery_ratio = len(ringo_clusters) / len(all_clusters) * 100
    # print(f'*Ringo* ratio for {molname} = {ringo_discovery_ratio:.2f}%')
    
    for method in methods:
        alt_clusters = set(df[df[f'{method}_found']]['cluster'].unique())
        alt_discovery_ratio = len(alt_clusters) / len(all_clusters) * 100
        print(f'{method} ratio for {molname} = {(alt_discovery_ratio - ringo_discovery_ratio):.2f}%')

## Analyze the conformers missed by Ringo and found by something else

In [ ]:
from diversity_analysis import get_diversity_df, METHOD_NAMES, PATHS

molname = 'csd_COHVAW'
diversity_analysis_settings = {
    'main_wd': './filtered_ensemble_diversity',
    'mode': 'filtered',
    'keep_unperspective': False,
    'keep_perspective': True,
    'use_refconformers': False,
    'plot_highenergy': True, 
    'plot_perspective': True,
    'expand_clusters': True,
    'merge_unclustered': True,
    'report_segments': ['reachability', 'diversity'],
}
if diversity_analysis_settings['main_wd'] not in PATHS['TOTALENSEMBLE_XYZ']:
    TOTALENSEMBLE_XYZ = os.path.join(diversity_analysis_settings['main_wd'], PATHS['TOTALENSEMBLE_XYZ'])
else:
    TOTALENSEMBLE_XYZ = PATHS['TOTALENSEMBLE_XYZ']

df, custom_order, key_names = get_diversity_df(molname, diversity_analysis_settings)

# Extract conformers generated by only RDKit
rdkit_df = df[(~df['unclustered']) & (df['Cluster located by'] == key_names['other'])]
rdkit_df = rdkit_df[rdkit_df['alt_method'].str.contains(METHOD_NAMES['rdkit'], regex=False)]
conf_idxs = rdkit_df['conf_id']
full_p = ringo.Confpool()
full_p.include_from_file(TOTALENSEMBLE_XYZ.format(molname=molname))
extracted_p = ringo.Confpool()
for idx in conf_idxs:
    extracted_p.include_from_xyz(full_p[idx].xyz, full_p[idx].descr)
extracted_p.atom_symbols = full_p.atom_symbols
extracted_p.save('check.xyz')

df['dihedral'] = df['conf_id'].apply(lambda conf_id: full_p[conf_id].z(20, 9, 10, 11)*180/3.1415926536)
df['isomer'] = df['dihedral'].apply(lambda dihedral: 'cis' if abs(dihedral) < 90.0 else 'trans')
from plotnine import ggplot, aes, ggtitle, geom_point, labs, facet_grid, theme_bw, element_text, theme, element_blank, element_rect, element_line, scale_size_manual, scale_shape_manual

plot_theme = (theme_bw() +
    theme(panel_grid_major = element_blank(),
    panel_grid_minor = element_blank(),
    panel_border = element_rect(colour="black", fill=None, size=1),
    axis_line = element_line(colour="black"),
    axis_title = element_text(size=12, face="bold"),
    axis_text = element_text(size=14),
    legend_title = element_text(size=14, face="bold"),
    legend_text = element_text(size=14),
    strip_text=element_text(size=14, face="bold"))
)

plot = ggplot(df, aes(x='x', y='y', color='isomer', shape='alt_method')) \
            + geom_point() + plot_theme + theme(figure_size=(12, 8))
plot

## Post-processing for accuracy analysis

In [ ]:
from environments import MAX_TIME

OLEG_DF_NAME = './experiment_compare.csv'
MAIN_MOLS = ['pdb_2QZK', 'csd_YIVNOG', 'pdb_3M6G', 'pdb_1NWX', 'pdb_2IYA', 'csd_RECRAT', 'csd_FINWEE10', 'csd_RULSUN', 'pdb_2C6H', 'csd_MIWTER']
MAIN_METHODS = ('rdkit', 'mtd', 'ringointel',)
df = pd.read_csv(OLEG_DF_NAME)
df = df.drop(columns=[
    'nconf',
    'nunique',
    # 'timelimit',
])
df = df.dropna()

df['n_uniquecorrect'] = df['n_uniquecorrect'].astype(int)
df = df[df['exp_rmsd'] < 2.5]
# df['timelimit'] = df['timelimit'].replace(MAX_TIME)
df = df.rename(columns={
    'time': 'time spent',
    'n_uniquecorrect': 'conformers generated',
    'exp_rmsd': 'Min CRMSD with experimental conformer',
})

method_type = CategoricalDtype(categories=[method_name for method_name in METHOD_NAMES.keys()], ordered=True)
df['method'] = df['method'].astype(method_type)
df.to_csv('experimental_rmsd.csv', index=False)

summary_df = pd.DataFrame(columns=['method', 'count', 'rmsd_threshold', 'timelimit'])
for timelimit in ('vshort', 'short', 'long'):
    # if timelimit == 'long':
    #     add_condition = df['testcase'].isin(MAIN_MOLS)
    # else:
    add_condition = True
    
    for rmsd_threshold in (0.5, 1.0):
        summary_part_df = df[ \
                (df['Min CRMSD with experimental conformer'] < rmsd_threshold) & \
                (df['timelimit'] == timelimit) & \
                add_condition \
            ] \
            .groupby(['method']) \
            .size() \
            .reset_index(name='count')
        if timelimit != 'long':
            summary_part_df = summary_part_df[summary_part_df['method'].isin(MAIN_METHODS)]
        summary_part_df['rmsd_threshold'] = rmsd_threshold
        summary_part_df['timelimit'] = timelimit
        summary_df = pd.concat([summary_df, summary_part_df])
summary_df.reset_index(drop=True, inplace=True)
print(summary_df)

from plotnine import ggplot, facet_grid, scale_fill_manual, aes, geom_point, geom_density, labs, scale_x_log10, scale_fill_manual, theme_bw, element_text, theme, element_blank, element_rect, element_line, geom_bar, position_dodge, ylim
normal_theme = (theme_bw() +
    theme(panel_grid_major = element_blank(),
        panel_grid_minor = element_blank(),
        panel_border = element_rect(colour='black', fill=None, size=1),
        axis_line = element_line(colour='black'),
        axis_title = element_text(size=16, face='bold', ma='center'),
        axis_text = element_text(size=14),
        legend_title = element_text(size=14, face='bold'),
        legend_text = element_text(size=14)
))
colorscheme = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#a65628", "#f781bf"]

cur_df = df[(df['time spent'] > 600.0) & df['testcase'].isin(MAIN_MOLS)]
plot = ggplot(cur_df, \
    aes(x='Min CRMSD with experimental conformer', fill='method')) + \
    normal_theme + \
    geom_density(alpha=0.8) + \
    facet_grid('method~.') + \
    scale_fill_manual(values=colorscheme) + \
    theme(figure_size=(8, 6))
print(plot)
plot.save(os.path.join(SI_DATA_DIR, f'experimental_allmethods.svg'), verbose=False)


plot = ggplot(df[(df['time spent'] > 600.0) & df['method'].isin(MAIN_METHODS)], \
    aes(x='Min CRMSD with experimental conformer', fill='method')) + \
    normal_theme + \
    geom_density(alpha=0.8) + \
    facet_grid('method~.') + \
    scale_fill_manual(values=colorscheme) + \
    theme(figure_size=(8, 3))
print(plot)
plot.save(os.path.join(SI_DATA_DIR, f'experimental_allmolecules_long.svg'), verbose=False)

plot = ggplot(df[(df['time spent'] < 600.0) & (df['time spent'] > 60.0) & df['method'].isin(MAIN_METHODS)], \
    aes(x='Min CRMSD with experimental conformer', fill='method')) + \
    normal_theme + \
    geom_density(alpha=0.8) + \
    facet_grid('method~.') + \
    scale_fill_manual(values=colorscheme) + \
    theme(figure_size=(8, 3))
print(plot)
plot.save(os.path.join(SI_DATA_DIR, f'experimental_allmolecules_short.svg'), verbose=False)

plot = ggplot(df[(df['time spent'] < 60.0) & df['method'].isin(MAIN_METHODS)], \
    aes(x='Min CRMSD with experimental conformer', fill='method')) + \
    normal_theme + \
    geom_density(alpha=0.8) + \
    facet_grid('method~.') + \
    scale_fill_manual(values=colorscheme) + \
    theme(figure_size=(8, 3))
print(plot)
plot.save(os.path.join(SI_DATA_DIR, f'experimental_allmolecules_vshort.svg'), verbose=False)